In [ ]:
pip install keras


In [ ]:
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l1_l2

# Load the dataset
data = pd.read_excel('/content/dataset.xlsx', engine='openpyxl')
# Define input and output columns
input_columns = [
    'Net Volume', 'Pulp Area', 'Froth surface area', 'Froth thickness', 'Air Flow rate',
    'R_inf Ccp', 'R_inf Gn', 'R_inf Po', 'R_inf Sp', 'k_max Ccp', 'k_max Gn', 'k_maxPo',
    'k_max Sp', 'Entrainement Savassi parameters', 'Total Solids Flow_Feed',
    'Total Liquid Flow_Feed', 'Pulp Volumetric Flow_Feed', 'Solids SG_Feed', 'Pulp SG_Feed',
    'Solids Fraction_Feed', 'Cu_Feed', 'Fe_Feed', 'Pb_Feed', 'Zn_Feed'
]

output_columns = [
    'Air Efficiency', 'Flotation Rate: Cell 1', 'Entrainment: Cell 1',
    'Total Solids Flow_Concentrate', 'Total Liquid Flow_Concentrate',
    'Pulp Volumetric Flow_Concentrate', 'Solids SG_Concentrate', 'Pulp SG_Concentrate',
    'Solids Fraction_Concentrate', 'Total Solids Flow_Tailings', 'Total Liquid Flow_Tailings',
    'Pulp Volumetric Flow_Tailings', 'Solids SG_Tailings', 'Pulp SG_Tailings',
    'Solids Fraction_Tailings', 'Cu_Tails', 'Fe_Tails', 'Pb_Tails', 'Zn_Tails',
    'Cu_Concentrate', 'Fe_Concentrate', 'Pb_Concentrate', 'Zn_Concentrate'
]

# Standard Scaling Features
scalerX = StandardScaler()
scaled_input = scalerX.fit_transform(data[input_columns])

# Scaling output data
scalerY = StandardScaler()
scaled_output = scalerY.fit_transform(data[output_columns])

# Splitting data into training and test sets
X_train, X_test, y_train, y_test = train_test_split(scaled_input, scaled_output, test_size=0.2, random_state=42)

# Model Definition Function
def create_model(optimizer='adam', neurons=64, dropout_rate=0.2, l1_rate=0.01, l2_rate=0.01):
    model = Sequential()
    model.add(Dense(neurons, activation='relu', input_shape=(X_train.shape[1],), kernel_regularizer=l1_l2(l1=l1_rate, l2=l2_rate)))
    model.add(BatchNormalization())
    model.add(Dropout(dropout_rate))
    model.add(Dense(neurons, activation='relu', kernel_regularizer=l1_l2(l1=l1_rate, l2=l2_rate)))
    model.add(Dense(len(output_columns)))
    model.compile(optimizer=optimizer, loss='mse')
    return model

# Parameters for grid search
param_grid = {
    'optimizer': ['adam', 'rmsprop'],
    'neurons': [50, 64, 100],
    'dropout_rate': [0.1, 0.2, 0.3],
    'l1_rate': [0.01, 0.001],
    'l2_rate': [0.01, 0.001]
}

# Custom loop for hyperparameter tuning
best_score = -1
best_params = None
for optimizer in param_grid['optimizer']:
    for neurons in param_grid['neurons']:
        for dropout_rate in param_grid['dropout_rate']:
            for l1_rate in param_grid['l1_rate']:
                for l2_rate in param_grid['l2_rate']:
                    model = create_model(optimizer, neurons, dropout_rate, l1_rate, l2_rate)
                    early_stopping = EarlyStopping(monitor='val_loss', patience=10)
                    model.fit(X_train, y_train, epochs=100, batch_size=32, validation_split=0.2, callbacks=[early_stopping])
                    y_pred = model.predict(X_test)
                    y_pred_rescaled = scalerY.inverse_transform(y_pred)
                    y_test_rescaled = scalerY.inverse_transform(y_test)
                    r2_scores = [r2_score(y_test_rescaled[:, i], y_pred_rescaled[:, i]) for i in range(y_test_rescaled.shape[1])]
                    average_r2 = sum(r2_scores) / len(r2_scores)
                    if average_r2 > best_score:
                        best_score = average_r2
                        best_params = (optimizer, neurons, dropout_rate, l1_rate, l2_rate)

print(f'Best R2 Score: {best_score}')
print(f'Best Parameters: {best_params}')

Le flux de sortie a été tronqué et ne contient que les 5000 dernières lignes.
Epoch 38/100
77/77 [==============================] - 0s 4ms/step - loss: 0.2469 - val_loss: 0.2080
Epoch 39/100
77/77 [==============================] - 0s 4ms/step - loss: 0.2450 - val_loss: 0.2061
Epoch 40/100
77/77 [==============================] - 0s 4ms/step - loss: 0.2537 - val_loss: 0.2120
Epoch 41/100
77/77 [==============================] - 0s 4ms/step - loss: 0.2447 - val_loss: 0.2026
Epoch 42/100
77/77 [==============================] - 0s 4ms/step - loss: 0.2382 - val_loss: 0.2048
Epoch 43/100
77/77 [==============================] - 0s 5ms/step - loss: 0.2387 - val_loss: 0.2044
Epoch 44/100
77/77 [==============================] - 0s 4ms/step - loss: 0.2457 - val_loss: 0.2131
Epoch 45/100
77/77 [==============================] - 0s 4ms/step - loss: 0.2456 - val_loss: 0.2008
Epoch 46/100
77/77 [==============================] - 0s 4ms/step - loss: 0.2376 - val_loss: 0.1957
Epoch 47/100
77/77 [==